# Efficient Python analysis with dynamic C++ and just-in-time compilation

Cppyy combines the convenience of the Python language with the efficiency of C++ implementations. The dynamic C++ bindings is powered by the C++ interpreter [cling](https://github.com/root-project/cling) allow to use conveniently efficient implementations in Python.

In [ ]:
import cppyy
import numpy as np

### Just-in-time compilation of C++ functions

In [ ]:
cppyy.cppdef('''

float smallest_diff(float* v1, float* v2, std::size_t size) {
    float min_diff = std::numeric_limits<float>::max();
    for (std::size_t i1 = 0; i1 < size; i1++) {
        for (std::size_t i2 = 0; i2 < size; i2++) {
            float diff = std::abs(v1[i1] - v2[i2]);
            if (diff < min_diff) {
                min_diff = diff;
            }
        }
    }
    return min_diff;
}
''');

As example inputs, we generate two numpy arrays with random numbers.

In [ ]:
size = 100
v1 = np.random.randn(size).astype(np.float32)
v2 = np.random.randn(size).astype(np.float32)

And next we benchmark the runtime:

In [ ]:
%%timeit
cppyy.gbl.smallest_diff(v1, v2, size)

How does the C++ kernel compare to a pure Python implementation?

In [ ]:
def smallest_diff(x1, x2):
    min_diff = float('inf')
    for e1 in x1:
        for e2 in x2:
            diff = abs(e1 - e2)
            if diff < min_diff:
                min_diff = diff
    return min_diff

**The Python implementation is a factor of 100 slower!**

In [ ]:
%%timeit
smallest_diff(v1, v2)

## Loading of precompiled functions

Improved C++ performance can be expected by precompiling the functionality and loading the library into cppyy 

In [ ]:
%%writefile diff_small.hxx

#include <cstddef>
#include <cmath> 
float optimized_smallest_diff(float* v1, float* v2, std::size_t size);

In [ ]:
%%writefile diff_small.cxx

# include "diff_small.hxx"

float optimized_smallest_diff(float* v1, float* v2, std::size_t size) {
    float min_diff = std::numeric_limits<float>::max();
    for (std::size_t i1 = 0; i1 < size; i1++) {
        for (std::size_t i2 = 0; i2 < size; i2++) {
            float diff = std::abs(v1[i1] - v2[i2]);
            if (diff < min_diff) {
                min_diff = diff;
            }
        }
    }
    return min_diff;
}


In [ ]:
!g++ -Ofast -shared -o libanalysis.so diff_small.cxx

Ex 3 - You can interactively include the header and functionality from the shared library.

In [ ]:
cppyy.cppdef('#include "diff_small.hxx"')
cppyy.load_library('libanalysis.so')

In [ ]:
cppyy.gbl.__dict__

**The loaded libary improves the runtime further!**

While this may not make a huge difference for small files like this example, large files benefit since your code is recompiled every time you `cppdef`

In [ ]:
%%timeit

cppyy.gbl.optimized_smallest_diff(v1, v2, size)

In [ ]:
cppyy.gbl.__dict__

Finally, we can show that all implementations come to the same result:

In [ ]:
print('Cppyy:', cppyy.gbl.smallest_diff(v1, v2, size))
print('Native Python:', smallest_diff(v1, v2))
print('Cppyy (loaded):', cppyy.gbl.optimized_smallest_diff(v1, v2, size))